### This notebook will extract the frames of a video and put them into another folder

In [1]:
import os
from pathlib import Path
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
ORIGINAL_VIDEO_DIRECTORY = Path("D:/__School/__Masters/____2021fALL/5280_aiwearables/videos_for_classification")
SINGLE_SUBJECT_DIR = Path("D:/__School/__Masters/____2021fALL/5280_aiwearables/videos_for_classification/AW")
SINGLE_VIDEO_DIR = Path("D:/__School/__Masters/____2021fALL/5280_aiwearables/videos_for_classification/AW/CLOCKWISE/WIN_20210913_17_35_36_Pro.MP4")
TEST_TARGET_DIR = Path("D:/__School/__Masters/____2021fALL/5280_aiwearables/test_target_dir")
GESTURE_NAMES = ["CLOCKWISE", "COUNTERCLOCKWISE", "DOWN", "UP", "LEFT", "RIGHT",] 

In [3]:
OUTPUT_DIR = Path("D:/__School/__Masters/____2021fALL/5280_aiwearables/class_videos/test")

In [4]:
def video_to_frames(input_dir, output_dir):
    """take in a video from input_dir and output the frames to the output_dir"""
    video = cv2.VideoCapture(str(input_dir))
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    for i in range(0, frame_count):
        success, image = video.read()
        if (image.size == 0):
            continue
        cv2.imwrite( str(output_dir) + f"\\img_{i+1:05d}.jpg", image)     # save frame as JPEG file

In [5]:
video_to_frames(SINGLE_VIDEO_DIR, OUTPUT_DIR)

In [6]:
def get_frame_rate(video_dir):
    """returns the frames_per_second of a recorded video"""
    frames_per_second = None
    video = cv2.VideoCapture(str(video_dir))
    frames_per_second = video.get(cv2.CAP_PROP_FPS)
    video.release()
    return frames_per_second

In [7]:
get_frame_rate(SINGLE_VIDEO_DIR)

15.220196539059657

In [8]:
def get_frame_count(video_dir):
    frame_count = None
    video = cv2.VideoCapture(str(video_dir))
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    video.release()
    return frame_count

In [9]:
get_frame_count(SINGLE_VIDEO_DIR)

46

In [10]:
def get_duration_seconds(video_dir):
    duration = None
    duration = get_frame_count(video_dir) / get_frame_rate(video_dir)
    return duration

In [11]:
get_duration_seconds(SINGLE_VIDEO_DIR)

3.0223

In [12]:
def get_dir_names_all_videos(home_dir):
    """returns the fps and length of every video in a directories subfolders"""
    video_fnames = []
    for path, subdirs, files in os.walk(home_dir):
        for name in files:
            video_fnames.append(str(Path(os.path.join(path, name))))
    return video_fnames

In [13]:
video_dir_names = get_dir_names_all_videos(ORIGINAL_VIDEO_DIRECTORY)

In [14]:
len(video_dir_names)

465

In [15]:
video_dir_names[1]

'D:\\__School\\__Masters\\____2021fALL\\5280_aiwearables\\videos_for_classification\\AW\\CLOCKWISE\\WIN_20210913_17_35_42_Pro.mp4'

In [16]:
video_dir_names[1].split("\\")[-3]

'AW'

In [17]:
video_dir_names[1].split("\\")[-2]

'CLOCKWISE'

In [18]:
video_dir_names[1].split("\\")[-1]

'WIN_20210913_17_35_42_Pro.mp4'

In [19]:
def get_metadata_all_videos(video_dir_names):
    fps_length_data = []
    
    for i, vid_dir in enumerate(video_dir_names):
        if i % 100 == 0:
            print(f"processing {i} of {len(video_dir_names)}...")
        split_dir = vid_dir.split("\\")
        fname = split_dir[-1]
        gesture_name = split_dir[-2].replace(" ", "").upper()
        subject_initials = split_dir[-3]
        
        video = cv2.VideoCapture(str(vid_dir))
        fps = video.get(cv2.CAP_PROP_FPS)
        frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        duration_seconds = frame_count / fps
        
        fps_length_data.append({"file_name": fname,
                                "subject_initials": subject_initials,
                                "gesture_name": gesture_name,
                                "frames_per_second": fps,
                                "frame_count": frame_count,
                                "duration_seconds": duration_seconds,
                                "dir_name": vid_dir})
    df = pd.DataFrame(fps_length_data)
    print("Done!")
    return df

In [20]:
df = get_metadata_all_videos(video_dir_names)

processing 0 of 465...
processing 100 of 465...
processing 200 of 465...
processing 300 of 465...
processing 400 of 465...
Done!


In [21]:
df

,file_name,subject_initials,gesture_name,frames_per_second,frame_count,duration_seconds,dir_name
0,WIN_20210913_17_35_36_Pro.mp4,AW,CLOCKWISE,15.220197,46,3.022300,D:\__School\__Masters\____2021fALL\5280_aiwear...
1,WIN_20210913_17_35_42_Pro.mp4,AW,CLOCKWISE,15.282554,44,2.879100,D:\__School\__Masters\____2021fALL\5280_aiwear...
2,WIN_20210913_17_35_47_Pro.mp4,AW,CLOCKWISE,15.269425,38,2.488633,D:\__School\__Masters\____2021fALL\5280_aiwear...
3,WIN_20210913_17_35_52_Pro.mp4,AW,CLOCKWISE,15.248184,38,2.492100,D:\__School\__Masters\____2021fALL\5280_aiwear...
4,WIN_20210913_17_35_56_Pro.mp4,AW,CLOCKWISE,15.213773,39,2.563467,D:\__School\__Masters\____2021fALL\5280_aiwear...
...,...,...,...,...,...,...,...
460,WIN_20211003_15_41_36_Pro.mp4,SP,UP,30.187100,101,3.345800,D:\__School\__Masters\____2021fALL\5280_aiwear...
461,WIN_20211003_15_41_42_Pro.mp4,SP,UP,30.279850,95,3.137400,D:\__School\__Masters\____2021fALL\5280_aiwear...
462,WIN_20211003_15_41_47_Pro.mp4,SP,UP,30.139188,110,3.649733,D:\__School\__Masters\____2021fALL\5280_aiwear...
463,WIN_20211003_15_41_53_Pro.mp4,SP,UP,30.165147,99,3.281933,D:\__School\__Masters\____2021fALL\5280_aiwear...


In [22]:
df["rounded_fps"] = df.frames_per_second.round()

In [23]:
df.rounded_fps.value_counts()

30.0    319
15.0     71
29.0     59
27.0      8
26.0      4
28.0      3
14.0      1
Name: rounded_fps, dtype: int64

In [24]:
df.describe()

,frames_per_second,frame_count,duration_seconds,rounded_fps
count,465.000000,465.000000,465.000000,465.000000
mean,27.530030,79.210753,2.866810,27.449462
std,5.325024,25.831489,0.701928,5.376566
min,14.445413,30.000000,1.009600,14.000000
25%,29.354439,49.000000,2.560467,29.000000
50%,29.973941,87.000000,2.966667,30.000000
75%,30.001429,99.000000,3.313567,30.000000
max,30.333219,133.000000,4.689567,30.000000


In [25]:
df.subject_initials.value_counts()

MCM    72
AW     60
JBG    60
KP     60
SP     60
SNB    51
IA     42
RAM    30
IM     12
RCS    12
KH      6
Name: subject_initials, dtype: int64

In [26]:
df.gesture_name.value_counts()

RIGHT               79
UP                  79
COUNTERCLOCKWISE    78
DOWN                78
LEFT                78
CLOCKWISE           73
Name: gesture_name, dtype: int64

In [27]:
df.frames_per_second.value_counts()

30.000000    53
30.001765     4
15.243902     3
29.965560     3
30.001456     3
             ..
29.989595     1
30.163444     1
29.763063     1
29.997551     1
30.203131     1
Name: frames_per_second, Length: 378, dtype: int64

In [28]:
df.frame_count.value_counts()

102    16
87     15
95     13
97     13
91     12
       ..
66      1
117     1
69      1
70      1
127     1
Name: frame_count, Length: 92, dtype: int64

In [29]:
df.duration_seconds.round().value_counts()

3.0    291
2.0     79
4.0     62
1.0     31
5.0      2
Name: duration_seconds, dtype: int64

In [37]:
def build_frames_directory(df, target_directory, gesture_names):
    """builds a directory for each class, and in each class a folder name"""
    
    def for_each_row(row):
        # make internal video directories
        gesture_dir = Path(target_directory).joinpath(row.gesture_name)
        video_dir = gesture_dir.joinpath(row.gesture_video_foldername)
        video_dir.mkdir(parents=True, exist_ok=True)
        
    for gesture_name in gesture_names:
        # make class directories
        gesture_directory = Path(target_directory).joinpath(gesture_name)
        gesture_directory.mkdir(parents=True, exist_ok=True)
    
    df.apply(for_each_row, axis=1)

In [31]:
def create_video_folder_names(df, gesture_names):
    """returns a dataframe with a new column formatted 
    <subject_initials>_<gesture_name>_<incremented_id>
    based on gesture classes"""
    df_list = []
    
    def for_each_row(row):
        # build video filename
        folder_name = f"{row.subject_initials}_{row.gesture_name}_{row.increment_id:05d}"
        return folder_name
    
    for gesture_name in gesture_names:
         # creates a new dataframe with increment values for each gesture
        dff = df[df["gesture_name"] == gesture_name].copy()
        dff["increment_id"] = np.arange(1, dff.shape[0]+1)
        dff["gesture_video_foldername"] = dff.apply(for_each_row, axis=1)
        df_list.append(dff)
    
        
    dff = pd.concat(df_list)
    dff.reset_index(drop=True, inplace=True)
    return dff

In [32]:
df = create_video_folder_names(df, GESTURE_NAMES)

In [33]:
df.head(1)

,file_name,subject_initials,gesture_name,frames_per_second,frame_count,duration_seconds,dir_name,rounded_fps,increment_id,gesture_video_foldername
0,WIN_20210913_17_35_36_Pro.mp4,AW,CLOCKWISE,15.220197,46,3.0223,D:\__School\__Masters\____2021fALL\5280_aiwear...,15.0,1,AW_CLOCKWISE_00001


In [41]:
df.dir_name[0]

'D:\\__School\\__Masters\\____2021fALL\\5280_aiwearables\\videos_for_classification\\AW\\CLOCKWISE\\WIN_20210913_17_35_36_Pro.mp4'

In [38]:
build_frames_directory(df, TEST_TARGET_DIR, GESTURE_NAMES)

In [46]:
def populate_frames(df, target_directory, gesture_names):
    """populate each of the video subfolders with individual frames from the videos"""
    def per_row(row):
        gesture_dir = Path(target_directory).joinpath(row.gesture_name)
        output_video_dir = gesture_dir.joinpath(row.gesture_video_foldername)
        input_video_dir = row.dir_name
        print(f"Processing {row.gesture_video_foldername}")
        video_to_frames(input_video_dir, output_video_dir)
        
    
    df.apply(per_row, axis=1)

In [ ]:
populate_frames(df, TEST_TARGET_DIR, GESTURE_NAMES)

Processing AW_CLOCKWISE_00001
Processing AW_CLOCKWISE_00002
Processing AW_CLOCKWISE_00003
Processing AW_CLOCKWISE_00004
Processing AW_CLOCKWISE_00005
Processing AW_CLOCKWISE_00006
Processing AW_CLOCKWISE_00007
Processing AW_CLOCKWISE_00008
Processing AW_CLOCKWISE_00009
Processing AW_CLOCKWISE_00010
Processing IA_CLOCKWISE_00011
Processing IA_CLOCKWISE_00012
Processing IA_CLOCKWISE_00013
Processing IA_CLOCKWISE_00014
Processing IA_CLOCKWISE_00015
Processing IA_CLOCKWISE_00016
